In [1]:
import numpy as np 
import pandas as pd
from tqdm import tqdm
import os
import pickle
from collections import defaultdict
import pickle
from os import walk
from numpy import linalg as LA
from collections import defaultdict

# Scripts

In [2]:
amino_list = [
    "ALA",
    "ARG",
    "ASN",
    "ASP",
    "CYS",
    "GLN",
    "GLU",
    "GLY",
    "HIS",
    "ILE",
    "LEU",
    "LYS",
    "MET",
    "PHE",
    "PRO",
    "PYL",
    "SER",
    "SEC",
    "THR",
    "TRP",
    "TYR",
    "VAL",
    "ASX",
    "GLX",
    "XAA",
    "XLE",
]

In [3]:

def parse_PDB(pdb_path, pdb_folder="protein_pdb_files/"):
    without_chain = False

    with open(pdb_path, "r") as fi:
        mdl = False
        for ln in fi:
            if ln.startswith("NUMMDL"):
                mdl = True
                break

    with open(pdb_path, "r") as fi:
        id = []

        if mdl:
            for ln in fi:
                if ln.startswith("ATOM") or ln.startswith("HETATM"):
                    id.append(ln)
                elif ln.startswith("ENDMDL"):
                    break
        else:
            for ln in fi:
                if ln.startswith("ATOM") or ln.startswith("HETATM"):
                    id.append(ln)

    count = 0
    seq = {}
    seq["type_atm"], seq["ind"], seq["amino"], seq["group"], seq["coords"] = (
        [],
        [],
        [],
        [],
        [],
    )

    for element in id:
        type_atm = element[0:6].strip().split()[0]
        ind = int(element[6:12].strip().split()[0])
        atom = element[12:17].strip().split()[0]
        amino = element[17:21].strip().split()[0]
        chain_id = element[21]
        group_id = int(element[22:26].strip().split()[0])
        x_coord = float(element[30:38].strip().split()[0])
        y_coord = float(element[38:46].strip().split()[0])
        z_coord = float(element[46:54].strip().split()[0])

        coords = np.array([x_coord, y_coord, z_coord])

        seq["type_atm"].append(type_atm)
        seq["ind"].append(int(ind))
        seq["amino"].append(amino)
        seq["group"].append(int(group_id))
        seq["coords"].append(coords)

        count += 1

    return seq["type_atm"], seq["amino"], seq["group"], seq["coords"]


In [4]:

def group_by_coords(group, amino, coords):
    uniq_group = np.unique(group)
    group_coords = np.zeros((uniq_group.shape[0], 3))

    group_amino = []

    np_group = np.array(group)

    #taking the amino acid coord as the mean position of the amino acid atoms
    for i, e in enumerate(uniq_group):
        inds = np.where(np_group == e)[0]
        group_coords[i, :] = np.mean(np.array(coords)[inds], axis=0)
        group_amino.append(amino[inds[0]])

    return group_coords, group_amino

In [5]:
def get_graph_from_struct(group_coords, group_amino, max_residues=2000):
    num_residues = group_coords.shape[0]

    if num_residues > max_residues:
        num_residues = max_residues

    residues = group_amino[:num_residues]

    retval = [[0 for i in range(0, num_residues)] for j in range(0, num_residues)]

    residue_type = []
    for i in range(0, num_residues):
        if residues[i] == "FME":
            residues[i] = "MET"
        elif residues[i] not in amino_list:
            residues[i] = "TMP"

        residue_type.append(residues[i])

        for j in range(i + 1, num_residues):
            x, y = group_coords[i], group_coords[j]
            retval[i][j] = LA.norm(x - y)
            retval[j][i] = retval[i][j]

    retval = np.array(retval)

    # threshold = 9.5 #struct2graph 
    threshold = 7

    for i in range(0, num_residues):
        for j in range(0, num_residues):
            if retval[i, j] <= threshold:
                retval[i, j] = 1
            else:
                retval[i, j] = 0

    n = retval.shape[0]
    adjacency = retval + np.eye(n)
    degree = sum(adjacency)
    d_half = np.sqrt(np.diag(degree))
    d_half_inv = np.linalg.inv(d_half)
    adjacency = np.matmul(d_half_inv, np.matmul(adjacency, d_half_inv))
    return residue_type, np.array(adjacency)


In [6]:

def create_fingerprints(atoms, adjacency, radius):
    fingerprints = []
    if (len(atoms) == 1) or (radius == 0):
        fingerprints = [fingerprint_dict[a] for a in atoms]
    else:
        for i in range(len(atoms)):
            vertex = atoms[i]
            neighbors = tuple(
                set(tuple(sorted(atoms[np.where(adjacency[i] > 0.0001)[0]])))
            )
            fingerprint = (vertex, neighbors)
            # print(fingerprint)
            fingerprints.append(fingerprint_dict[fingerprint])
    # print(fingerprints)
    return np.array(fingerprints)

In [7]:
def dump_dictionary(dictionary, file_name):
    with open(file_name, "wb") as f:
        pickle.dump(dict(dictionary), f)

# loop

In [72]:
protein_pdb_folder = "protein_pdbs"
protein_fp_folder = "protein_fingerprints"
adj_folder = os.path.join(protein_fp_folder,"adj")
fp_folder = os.path.join(protein_fp_folder,"fingerprint")
protein_pdb_files = [file for file in os.listdir(protein_pdb_folder) if file.endswith(".pdb")]

In [73]:
# transfer protein into fingerprint
fingerprint_dict = defaultdict(lambda: len(fingerprint_dict))
aa_dict = defaultdict(lambda: len(aa_dict))
p_names, adjacencies, fingerprints = [],[],[]
sample_protein = protein_pdb_files[0]
for sample_protein in tqdm(protein_pdb_files):
    uniprot_id = sample_protein.split(".")[0]
    # print(f"processing {uniprot_id}")
    pdb_path = os.path.join(protein_pdb_folder, sample_protein)
    type_atm, amino, group, coords = parse_PDB(pdb_path)
    group_coords, group_amino = group_by_coords(group, amino, coords)
    residue_type,adjacency = get_graph_from_struct(group_coords, group_amino)
    aminos = np.array([aa_dict[a] for a in residue_type])
    # print(f"aminos={aminos}")
    fingerprint = create_fingerprints(aminos, adjacency, 1)
    # print(f"fingerprints: {fingerprint}")
    p_names.append(uniprot_id)
    adjacencies.append(adjacency)
    fingerprints.append(fingerprint)
    # transfer protein into adj list 
    np.save(os.path.join(adj_folder,uniprot_id),np.array(adjacency,dtype=object),allow_pickle=True)
    # transfer protein into fingerprint list 
    np.save(os.path.join(fp_folder,uniprot_id),np.array(fingerprint,dtype=object),allow_pickle=True)


100%|██████████| 361/361 [03:24<00:00,  1.76it/s]


In [68]:
adjacencies = np.array(adjacencies, dtype=object)
fingerprints = np.array(fingerprints, dtype=object)

In [69]:
np.save(os.path.join(protein_fp_folder, "protein_names.npy"), np.array(p_names), allow_pickle=True)
np.save(os.path.join(protein_fp_folder, "protein_adjacencies.npy"), np.array(adjacencies), allow_pickle=True)
np.save(os.path.join(protein_fp_folder, "protein_fingerprints.npy"), np.array(fingerprints), allow_pickle=True)

In [ ]:
dump_dictionary(fingerprint_dict,os.path.join(protein_fp_folder,"prot_fingerprint_dict.pickle"))

# debug

In [54]:
test_adj = np.load("protein_fingerprints/old/protein_adjacencies.npy",allow_pickle=True)


In [57]:
np.save("test_adj.npy",adjacecnies)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (361,) + inhomogeneous part.

In [70]:
test_dict = np.load(os.path.join(protein_fp_folder,"fingerprint_dict.pickle"),allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'protein_fingerprints/fingerprint_dict.pickle'

In [19]:
print("threshold = 7")
len(fingerprint_dict)

threshold = 7


59238

In [16]:
fingerprints[200]

array([ 2937,  6674,   729,   503,  2397,  2375,  1803,  1861,  4245,
        1861,  5642,   503,   608,  7253,  1226,  1307,  1327,  4335,
        5064,  5648, 42649,  1828, 10680,   392, 17999,  2822,  4584,
       17663,  1442, 16858,  8057,  4992, 42650,  5542, 21056, 42651,
       42652, 42653, 11230, 20433, 42654,  1236, 42655,  4803,  1302,
        4164,  4548,  1303, 10063, 42656,  1255,  2939,  4965,  8998,
        8997, 42657,   452,  8405, 39169, 16114, 42658, 42659, 42660,
       42661, 42662, 42663, 42664, 42665, 42666, 42667, 11562,    79,
         824, 42668, 19181, 11517, 42669, 42670, 42671, 42672, 42673,
       42674, 42675, 38748,  2454, 42676, 42677, 42678, 42679, 42680,
       42681, 42682, 42683, 42684, 42685, 42686,  5632, 21160, 42687,
       42688, 42689, 42690, 42691, 42692, 42693, 42694, 42695, 42696,
       42697, 42698, 42699,   522, 10898,   503,  2527, 42700, 42701,
       42702, 42703, 42704, 42705, 42706, 15948, 22292, 42313,   503,
       11543, 42314,

In [24]:
# test_dict = np.load(os.path.join(protein_fp_folder,"protein_fingerprints.npy"),allow_pickle=True)

In [29]:
test_dict = np.load("protein_fingerprints/fingerprint_dict.pickle",allow_pickle=True)

In [30]:
test_dict

{(0, (2,)): 0,
 (0, (2, 2)): 1,
 (1, (2, 2)): 2,
 (2, (2, 2, 2)): 3,
 (2, (2,)): 4,
 (2, (2, 2)): 5,
 (1, (2, 2, 2)): 6,
 (1, (2,)): 7,
 (2, (2, 2, 2, 2)): 8,
 (3, (0, 0, 0, 0)): 9,
 (3, (1, 2)): 10,
 (3, (2, 2)): 11,
 (1, (1, 2, 2)): 12,
 (4, (2,)): 13,
 (0, (1, 2)): 14,
 (1, (2, 2, 5)): 15,
 (3, (2,)): 16,
 (6, (2,)): 17,
 (7, (2,)): 18,
 (8, (2,)): 19,
 (1, (2, 2, 2, 2)): 20,
 (1, (1,)): 21,
 (1, (1, 1, 2)): 22,
 (0, (1,)): 23,
 (0, (1, 1)): 24,
 (9, (0, 0, 0, 2)): 25,
 (9, (2,)): 26,
 (3, (1, 1)): 27,
 (9, (0, 0, 0, 0)): 28,
 (9, (0,)): 29,
 (1, (1, 2)): 30,
 (3, (3, 2)): 31,
 (3, (0, 2)): 32,
 (1, (1, 1)): 33,
 (3, (0, 0, 2, 2)): 34,
 (9, (1,)): 35,
 (3, (0,)): 36,
 (1, (1, 1, 1, 2)): 37,
 (3, (3,)): 38,
 (3, (0, 0, 1, 2)): 39,
 (3, (0, 0, 0, 2)): 40,
 (3, (9, 9)): 41,
 (9, (0, 0, 0)): 42}

In [10]:
prots = []
protein_folder = "protein_pdbs"
protein_fingerprints_folder = "protein_fingerprints"
for file in os.listdir(protein_folder):

SyntaxError: incomplete input (4064245176.py, line 4)